In [56]:
import numpy as np
import pandas as pd
from fractions import Fraction

In [57]:
NUM_PLAYERS = 3
PLAYERS = list(range(1, NUM_PLAYERS+1))
# all possible (giver, receiver) pairs
OUTCOME_PROB = Fraction(1, NUM_PLAYERS * (NUM_PLAYERS-1))  # 1/6
CANDIDATE_MOVES = [(g, r) for idx, g in enumerate(PLAYERS) for r in PLAYERS if g != r]
LOSER_IDX = 0

# Helper functions

## Functional

In [59]:
def applyMove(initState, move):
    """
    Applies the given move to the initial state:
    - initState: an integer triple representing the initial wealth of each player
    - move: a (giver, receiver) pair
    Returns the resulting wealth of the players as a triple.
    """
    g, r = move
    initWealth = list(initState)
    transferAmt = min(initWealth[g-1], initWealth[r-1])  # 0-indexed
    initWealth[g-1] -= transferAmt  # give
    initWealth[r-1] += transferAmt  # receive
    return tuple(initWealth)


def getChildStates(initState):
    """
    Applies all possible moves to the initial state.
    Returns the child states.
    """
    return [applyMove(initState, move) for move in CANDIDATE_MOVES]

def getCoef(initState):
    """
    Returns a pair consisting of:
    - a dictionary of (state:coefficient) pairs, representing the coefficient of the hitting probability 
      for each state in the first-step analysis, starting from the given initial state
    - a rational number representing the constant in the equation.
    """
    # assign all states the individual outcome probability
    childStates = getChildStates(initState)
    stateCoefs = {s:OUTCOME_PROB for s in childStates}
    const = 0
    
    # apply any possible simplifications to the states
    copy = stateCoefs.copy()
    for state in copy:
        # prune the 0 hitting prob states
        if zeroHittingProb(state):
            stateCoefs.pop(state)
        # convert the hitting prob 1 states to constants
        elif alreadyHit(state):
            const += stateCoefs[state]
            stateCoefs.pop(state)
        # optimsation 1: convert equal wealth states to constants
        elif equalWealth(state):
            const += stateCoefs[state] / NUM_PLAYERS
            stateCoefs.pop(state)
    
    # optimization 2: combine symmetrical states
    copy = list(stateCoefs.copy().keys())
    n = len(copy)
    for i in range(n):
        for j in range(i+1, n):
            s1, s2 = copy[i], copy[j]
            if (symmetricalStates(s1, s2)):
                combinedProb = stateCoefs.pop(s1) + stateCoefs.pop(s2)
                stateCoefs[standardiseState(s1)] = combinedProb
    
    return standardiseStates(stateCoefs), const

def zeroHittingProb(state):
    """
    Returns true if the given state cannot reach the state of Player 1 being the loser, false otherwise.
    """
    return (0 in state) and (state[LOSER_IDX] > 0)

def alreadyHit(state):
    """
    Returns true if the given state reaches the state of Player 1 being the loser wp1, false otherwise.
    """
    return state[LOSER_IDX] == 0

def symmetricalStates(s1, s2):
    """
    Returns true if the two states are symmetrical (undistinguished apart from the loser), false otherwise.
    """
    return s1[LOSER_IDX] == s2[LOSER_IDX] and set(s1[LOSER_IDX+1:]) == set(s2[LOSER_IDX+1:])
    
def equalWealth(s):
    """
    Returns true if all players have the same wealth in the current state (an integer triple), 
    false otherwise.
    """
    return all(amt == s[0] for amt in s)

## Formatting / Representational

In [60]:
from sympy import *
from sympy import nsolve
from sympy.solvers import solve
from ast import literal_eval as make_tuple  # for parsing state as a tuple

In [61]:
def standardiseState(s):
    """
    Returns the standardised form of the state (a triple).
    The convention is to keep the smaller value of the undistinguished players at the front.
    """
    return (s[LOSER_IDX],) + tuple(sorted(s[LOSER_IDX+1:]))

def standardiseStates(stateCoefs):
    """
    Standardises all states (keys) in the dictionary of (state:coefficient) pairs.
    Returns the standardised form of the dictionary.
    """
    return {standardiseState(s):p for s, p in stateCoefs.items()}

def getEquation(initState, stateCoefs, const):
    """
    Converts the dictionary of state:coefficient pairs and constant
    into an equality of the form: h(initState) = coefs * h(states) + const
    """
    # equation will be in the form of human readable equalities (like how we normally write down)
    symbols = [stateToSymbol(s) for s in stateCoefs]
    eqn = Eq(stateToSymbol(initState), getExpr(stateCoefs, const))
    return eqn, symbols
    
def stateToSymbol(state):
    """
    Converts the state (triple) to a Symbol object.
    """
    return Symbol(f'h{state}')

def symbolToState(symbol):
    """
    Converts the Symbol object to a state (triple).
    """
    return make_tuple(str(symbol)[1:])

def getExpr(stateCoefs, const):
    """
    Converts the dictionary of state:coefficient pairs and constant
    into an expression.
    """
    expr = const
    for s in stateCoefs:
        expr = Add(expr, stateCoefs[s] * stateToSymbol(s))
    return expr

def formatEquation(eqn):
    """
    Returns the formatted string for the equation.
    """
    return f'{eqn.lhs} = {eqn.rhs}'

def formatSolutions(sol):
    """
    Returns the solutions as a formatted list (states converted back to triples)
    """
    return [(symbolToState(s), p) for s, p in sol.items()]

# Main class for the first-step analysis

In [62]:
class LoserAnalysis():
    def __init__(self, initState):
        self.initState = standardiseState(initState)
        self.generatedStates = set([initState])
        self.equations = []
        self.symbols = set([stateToSymbol(self.initState)]) # or can encode all generated states at once in the end
    
    def getEquations(self):
        """
        Returns a list of equations (as Equality objects) generated during the first step analysis.
        """
        self.recurseGenerateEqns(self.initState)
        return self.equations
        
    def recurseGenerateEqns(self, initState):
        """
        Recursively generates the first step analysis equations,
        and records the new symbols and equations.
        """
        childStatesProb, const = getCoef(initState)  # state:prob pairs, under unified representation
        newStates = set(childStatesProb).difference(self.generatedStates)
        
        # compute for the current state
        # do this before the recursion to close off the loop
        eqn, symbols = getEquation(initState, childStatesProb, const)
        self.equations.append(eqn)

        # base case: no new states
        if not newStates:
            return

        self.symbols = self.symbols.union(symbols)
        self.generatedStates = self.generatedStates.union(newStates)
        
        # recurse
        for state in newStates:
            self.recurseGenerateEqns(state)
    
    def solveEqns(self):
        """
        Returns the hitting probabilities of all states involved in the analysis,
        as a dictionary of (state symbol : probability) pairs.
        The probabilities are represented as fractions.
        """
        self.getEquations()
        return solve(self.equations, self.symbols)
    
    def solveEqnsApprox(self):
        """
        Similar to `solveEqns()`, but the hitting probabilities are represented as floating point numbers.
        """
        sol = self.solveEqns()
        return {s:float(p) for s, p in sol.items() if sol}
    
    def exportEqns(self, filename):
        """
        Exports the first step analysis equations to a text file, 
        using the provided filename.
        """
        self.getEquations()
        fp = open(filename, 'w')
        for e in self.equations:
            fp.write(formatEquation(e) + '\n')
        fp.close()

## Examples

In [63]:
LoserAnalysis((1,2,3)).solveEqnsApprox()

{h(1, 1, 4): 0.4731182795698925,
 h(1, 2, 3): 0.5207373271889401,
 h(2, 1, 3): 0.31797235023041476,
 h(3, 1, 2): 0.16129032258064516,
 h(4, 1, 1): 0.053763440860215055}

In [64]:
LoserAnalysis((5,7,8)).solveEqns()

{h(1, 1, 18): 42158656716086853639363655382735325877/84592376193700184596241343286179508524,
 h(1, 2, 17): 4284827905625134243422857147044029735919/7303141811389449270142169303706830902572,
 h(1, 3, 16): 35548295235852279099190454831911842523/57961442947535311667794994473863737322,
 h(1, 4, 15): 93175618338117767214446626573160714201/149043710436519372860044271504221038828,
 h(1, 5, 14): 650006532286273868033229287193845033119/1043305973055635610020309900529547271796,
 h(1, 6, 13): 36701667407853840658020228136032587789/57961442947535311667794994473863737322,
 h(1, 7, 12): 3966980956662554030252875664848298613847/6259835838333813660121859403177283630776,
 h(1, 8, 11): 422131361925067975033171969473812147/671368065029366544414613835604599274,
 h(1, 9, 10): 3714777279572669611218001904679583184203/6259835838333813660121859403177283630776,
 h(10, 1, 9): 3762814143770034654518082859356383029/24840618406086562143340711917370173138,
 h(10, 2, 8): 3838312/24345489,
 h(10, 3, 7): 1203031268306

In [44]:
LoserAnalysis((5,7,8)).exportEqns('578Equations.txt')

# Aggregate states and hitting probs to one file

In [69]:
import csv

In [70]:
def multipleStateProbs(minWealth, maxWealth):
    """
    Returns the P(Loser = Player 1) analysis solutions as a dictionary of (state:hitting probability) pairs, 
    beginning from all possible initial states, 
    where each player's amount is within [`minWealth`, `maxWealth`] (inclusive).
    """
    stateProbs = dict()  # states as symbols : hitting prob

    for x in range(minWealth, maxWealth+1):
        for y in range(minWealth, maxWealth+1):
            for z in range(minWealth, maxWealth+1):
                initState = (x, y, z)
                sols = LoserAnalysis(initState).solveEqns()
                if sols:
                    newStates = set(sols).difference(stateProbs)
                    for s in newStates:
                        stateProbs[s] = sols[s]

    # sort by state
    return dict(sorted(formatSolutions(stateProbs)))

In [ ]:
def exportStateProbs(filename, minWealth, maxWealth):
    """
    Creates a CSV file with columns (Initial state, P(Loser = Player 1)).
    The state is an integer triple, and the probability is a rational number.
    - filename: a string representing the filename of the CSV file;
    - minWealth: the minimum allowable wealth for the range of initial states simulated;
    - maxWealth: the maximum allowable wealth for the range of initial states simulated.
    """
    
    stateProbs = multipleStateProbs(minWealth, maxWealth)
    
    fp = open(FILENAME, 'w')
    writer = csv.writer(fp)
    writer.writerow(['Initial state', 'P(Loser = Player 1)'])
    writer.writerows(stateProbs.items())
    fp.close()

In [41]:
exportStateProbs('Player1LoseProbs_1to5.csv')